In [441]:
# Core libraries
import os
import io
import glob
import warnings
from collections import defaultdict
import contextlib

# Numerical and scientific tools
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm

# Plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter

# Astropy: units, constants, FITS I/O
import astropy.units as u
import astropy.constants as c
from astropy.constants import R_sun
from astropy.io import fits


# Spectral tools
from specutils import Spectrum
from specutils.manipulation import FluxConservingResampler

# Progress bars
from tqdm import tqdm




In [442]:
#restframe='TOI5205/Rest Frames/StellarRestFrame_202204200018.fits'
#restframe='TOI5205/Rest Frames/StellarRestFrame_202204200019.fits' 
restframe='TOI5205/Rest Frames/StellarRestFrame_202204210023.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202204210024.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202204290024.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202204290025.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202205020057.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202205020058.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202205130036.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202205130037.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202205140042.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202205140043.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202205190035.fits' 
#restframe='TOI5205/Rest Frames/StellarRestFrame_202205190036.fits'
#restframe='TOI5205/Rest Frames/StellarRestFrame_202204200018.fits'
folder_path = 'TOI5205/Rest Frames'
parq_file = 'TOI5205/lte034-5.0-0.0a+0.0.BT-Cond.7.parquet'
lfc = "Laser Frequency Comb/LFC_1D.fits" #if change the LFC file, will need to change the parameters to cut out the laser in injection_laser_flux and injection_laser_variance

def constants_to_change():
    
    model_shift = 2.7*u.Angstrom #how much the model is shifted to the right in angstroms to fit the observed data shape
    
    #STAR PARAMETERS
    star_radius = 0.394 * R_sun
    distance_to_star = 86.4493 * u.parsec
    v_rot_kms = 2.0 * u.km / u.s
    
    #LASER INJECTION PARAMETERS
    
    laser_center = 10010  # in Angstroms
    injection_laser_power = 0.5* u.MW
    
    minimum_snr = 5 # this is the SNR above the baseline (average SNR of the observed data) that we want to detect the laser signal
    transmitter_diameter = 10 * u.m  # assuming diameter of transmitting telescope is 10 m
    telescope_radius = 4.25 * u.m  # radius of receiving telescope (HPF has diameter of 8.5 m)
    return{
            "model shift": model_shift,
            "star radius": star_radius,
            "distance to star": distance_to_star,
            "v rot": v_rot_kms,
            "laser center": laser_center,
            "injection laser power": injection_laser_power,
            "minimum snr": minimum_snr,
            "transmitter diameter": transmitter_diameter,
            "telescope radius": telescope_radius
        }
    

In [443]:
#broadening the model spectrum based on the rotational velocity of the star looking at, which can be changed in constants to change
def apply_rotation_broadening(spectrum: Spectrum):
    from scipy.ndimage import gaussian_filter1d
    import numpy as np
    import matplotlib.pyplot as plt
    v_rot_kms = constants_to_change()["v rot"]  # rotational velocity in km/s
    
    #gaussian rotation broadening
    c_kms = c.c.to(u.km / u.s)  # speed of light in km/s
    wl = spectrum.spectral_axis.to(u.Angstrom)
    flux = spectrum.flux

    #gaussian width in angstroms
    sigma = (v_rot_kms / c_kms) * wl
    #use mean sigma per pixel
    sigma_pix = np.mean(sigma) / np.diff(wl).mean()

    broadened_flux = gaussian_filter1d(flux, sigma_pix)

    return Spectrum(spectral_axis=spectrum.spectral_axis, flux=broadened_flux * spectrum.flux.unit)

In [444]:
# opening observation fits file, specifying wavelengths observed, subtracting sky background, and plotting the observed spectrum
def obs_spectrum(im):
    hdul = fits.open(im)
    sci_obs = hdul[1].data 
    sky_obs = hdul[2].data
    flux_obs = sci_obs - sky_obs
    variance = hdul[4].data
    wavelength_obs = hdul[10].data  # shape of (n_orders, n_pixels) in Angstroms
    n_orders = wavelength_obs.shape[0]
    
    #masking for wavelength range and nan/negative values
    mask_wave = (wavelength_obs > 8000) & (wavelength_obs < 13000) 
    mask_nan = ~np.isnan(flux_obs) 
    mask_positive = (flux_obs > 0)
    combined_mask = mask_nan & mask_positive & mask_wave #combine all of the masks into one
    mask = combined_mask
    
    #plotting full observed spectrum
    # plt.figure(figsize=(10, 6))
    # plt.plot(wavelength_obs[mask], flux_obs[mask], label='Observed Spectrum')
    # plt.xlabel("Wavelength (Angstroms)")
    # plt.ylabel("Photons / second")
    # plt.title(f"Observed Spectrum: {os.path.basename(hdul.filename())}")
    # plt.legend()
    # plt.grid(True)
    # plt.show()
    
    orders = []

    for i in range(n_orders):
        wl = wavelength_obs[i]
        flux = flux_obs[i]

        # Apply masks
        mask = (
            (wl > 8000) & (wl < 13000) & 
            ~np.isnan(flux) & (flux > 0)
        )

        wl_clean = wl[mask] * u.Angstrom
        flux_clean = flux[mask]

        # calculate bin widths
        binwidths = np.diff(wl_clean.value)
        binwidths = np.append(binwidths, binwidths[-1]) * u.Angstrom

        if len(wl_clean) == 0:
            continue  # skip empty or fully masked orders

        orders.append({
            "order": i + 1,
            "wavelength": wl_clean,
            "flux": flux_clean,
            "binwidths": binwidths,
            "variance": variance[i][mask] if i < len(variance) else None
        })
    
    hdul.close()
    return orders

In [445]:
# masking out anything not in the range of the observed wavelengths, changing flux into photons/ second, resampling based on the observed bin widths
def plot_model_flux_with_obs_bins(table_path, order):
    obs_wavelengths = order["wavelength"]
    obs_binwidths = order["binwidths"]
    wl_min = obs_wavelengths.min().value
    wl_max = obs_wavelengths.max().value
    # Load model data and filter by wavelength
    model_table = pd.read_parquet(table_path)
    mask = (model_table['wavelength'] >= wl_min) & (model_table['wavelength'] <= wl_max)
    filtered_table = model_table[mask]

    model_wavelength = filtered_table['wavelength'].values * u.Angstrom
    model_flux = filtered_table['flux'].values * u.erg / (u.cm**2 * u.s * u.Angstrom)

    #creating spectrum 1D object to do broadening on
    model_spectrum = Spectrum(spectral_axis=model_wavelength, flux=model_flux)

    #rotational broadening
    broadened_model = apply_rotation_broadening(model_spectrum)

    model_shift = constants_to_change()["model shift"]  # in Angstroms
    shifted_wavelength = broadened_model.spectral_axis + model_shift
    shifted_model = Spectrum(spectral_axis=shifted_wavelength, flux=broadened_model.flux)

    Ep = (c.h * c.c / obs_wavelengths.to(u.m)).to(u.J)

    radius_star = constants_to_change()["star radius"]  # in meters
    distance_to_star = constants_to_change()["distance to star"]  # in parsecs
    telescope_radius = constants_to_change()["telescope radius"]  # in meters
    Area_tel = np.pi * (telescope_radius)**2

    # Resample model onto observed wavelength bins
    resampler = FluxConservingResampler()
    resampled_model = resampler(shifted_model, obs_wavelengths)

    # Convert model flux to photon flux
    model_flux_interp = resampled_model.flux
    photon_flux = (model_flux_interp * (radius_star/distance_to_star)**2 * Area_tel * obs_binwidths / Ep).to(1 / u.s)

    return obs_wavelengths, photon_flux

In [446]:
# taking a laser line flux from LFC 1D
def injection_laser_flux(file_path): # some numbers hard coded because using same laser for all injections for convenience
    hdul = fits.open(file_path)
    flux_array = hdul[1].data  # (28, 2048)
    order_index = 13  # Only plot one order, this is order + 1
    injection_flux = flux_array[order_index]-3.3589930125 #subtract off average of both sides of the laser peak
    mask1 = (np.arange(len(injection_flux)) < 1370)
    injection_flux[mask1] = 0
    mask2 = (np.arange(len(injection_flux))) > 1374
    injection_flux[mask2] = 0
    
    wavelength=hdul[7].data[order_index]  # Wavelengths for the order
    
    return wavelength, injection_flux
# laser line variance from LFC 1D
def injection_laser_variance(file_path): # this is one fits file and one fiber only
    hdul = fits.open(file_path)
    variance_array = hdul[4].data  # (28, 2048)
    order_index = 13  # Only plot one order, this is order + 1
    injection_variance = variance_array[order_index]-0.0025471 #subtract off average of both sides of the laser peak
    mask1 = (np.arange(len(injection_variance)) < 1370)
    injection_variance[mask1] = 0
    mask2 = (np.arange(len(injection_variance))) > 1374
    injection_variance[mask2] = 0
    wavelength=hdul[7].data[order_index] 
    
    return wavelength,injection_variance

In [447]:
def generate_laser_line(
    wavelength_center=10000,     # Å
    laser_power_kW=1,           # Laser power in kW
    pixel_width_angstrom=0.2,    # Å/pixel
    fwhm_pixels=1.65,            # Gaussian FWHM in pixels
    n_sigma=5                    # How wide to simulate the Gaussian (in sigma)
):
    """
    Simulate a Gaussian laser line scaled to a given laser power.
    
    Parameters:
    - wavelength_center: Central wavelength of laser (Å)
    - laser_power_kW: Power of laser (kW)
    - pixel_width_angstrom: Å per pixel in spectrum
    - fwhm_pixels: Gaussian width in pixels (default 1.65)
    - n_sigma: Number of sigma to simulate around center (default ±5σ)
    
    Returns:
    - wavelength [Å]
    - flux [photons/s/Å]
    - variance [photons²/s²/Å²]
    """

    # Convert power to photon flux
    laser_power_W = (laser_power_kW * 1e3) *u.W  # W
    lambda_m = wavelength_center * 1e-10  # Å → m
    E_photon = c.h * c.c / lambda_m           # Energy per photon [J]
    photon_flux_total = laser_power_W / E_photon  # photons/s

    # Convert FWHM from pixels to Ångström
    fwhm_angstrom = fwhm_pixels * pixel_width_angstrom
    sigma_angstrom = (fwhm_angstrom / 2.355)

    # Generate wavelength grid around laser line
    wave_min = wavelength_center - n_sigma * sigma_angstrom
    wave_max = wavelength_center + n_sigma * sigma_angstrom
    num_points = int((wave_max - wave_min) / pixel_width_angstrom) + 1
    wavelength = np.linspace(wave_min, wave_max, num_points)

    # Create normalized Gaussian profile
    profile = norm.pdf(wavelength, loc=wavelength_center, scale=sigma_angstrom)
    profile /= np.sum(profile)  # Normalize to unit area

    # Scale profile by total photon flux to get specific flux (photons/s/Å)
    flux = profile * photon_flux_total

    # Poisson noise: variance = flux
    variance = flux.copy()

    return wavelength, flux, variance

In [448]:
# injecting laser line into the observed spectrum at the wavelength specified
# then scales laser flux and variance until finds minimum scaling factor that results in at least 5 pixels with SNR excess above baseline of 5
def inject_and_analyze_laser(orders, injection_flux, injection_variance, wavelength_injection,
                              median_eff_per_order, parq_file=None):
    desired_laser_center = constants_to_change()["laser center"] * u.Angstrom
    original_laser_peak = wavelength_injection[np.argmax(injection_flux)] * u.Angstrom
    shift_angstroms = (desired_laser_center - original_laser_peak).to(u.Angstrom).value
    shifted_wavelength_injection = wavelength_injection + shift_angstroms

    full_wavelength = []
    full_injected_flux = []
    full_injected_variance = []
    full_model_flux = []
    full_residual = []
    full_snr = []
    full_snr_excess = []

    all_excess_snr = []
    all_laser_wavelengths = []
    all_laser_excess = []

    # Load model if missing
    for order_index, order in enumerate(orders):
        if "model_photon_flux" not in order:
            model_wavelength, model_photon_flux = plot_model_flux_with_obs_bins(parq_file, order)
            order["model_wavelength"] = model_wavelength
            order["model_photon_flux"] = model_photon_flux

    # Binary search for minimum scaling factor
    def laser_detected(scale):
        detected_pixels = 0
        for order_index, order in enumerate(orders):
            wl_obs = order["wavelength"].value
            obs_flux = order["flux"]
            obs_variance = order["variance"]
            model_flux = order["model_photon_flux"].value * median_eff_per_order[order_index]
            minimum_snr = constants_to_change()["minimum snr"]
            
            #laser_flux_interp = np.interp(wl_obs, shifted_wavelength_injection, injection_flux * scale, left=0, right=0)
            laser_flux_interp = np.interp(wl_obs, shifted_wavelength_injection, injection_flux * scale, left=0, right=0) 
            laser_var_interp = np.interp(wl_obs, shifted_wavelength_injection, injection_variance * scale, left=0, right=0) 



            injected_flux = obs_flux + laser_flux_interp
            injected_variance = obs_variance + laser_var_interp #mentioned snr doesn't use variance so do I need to include this?

            residual_flux = injected_flux - model_flux
            residual_variance = injected_variance / np.sqrt(model_flux)
            snr = residual_flux / np.sqrt(residual_variance) #ASK ABOUT SNR CALCULATIONS: is it just np.sqrt(residual_flux)?
            
            
            snr_excess = snr 
            #laser_pixels are 2 below and above laser peak
            #laser_pixels = np.where((wl_obs >= desired_laser_center.value - 2) & (wl_obs <= desired_laser_center.value + 2))[0]
            laser_pixels = np.where(laser_flux_interp > 0)[0]
            snr_excess_laser = snr_excess[laser_pixels]
            detected_pixels += np.sum(snr_excess_laser >= minimum_snr)
            #detected_pixels += np.sum(snr >= minimum_snr)

        return detected_pixels >= 5

    # Search scaling factor between 1e-6 and 1.0
    min_scale, max_scale = 1e-6, 1.0
    for _ in range(20):  # binary search
        mid_scale = (min_scale + max_scale) / 2
        if laser_detected(mid_scale):
            max_scale = mid_scale
        else:
            min_scale = mid_scale
    laser_scale = max_scale

    #print(f"Minimum Laser Scaling Factor for Detection: {laser_scale:.6f}")
    injection_flux *= laser_scale
    injection_variance *= laser_scale
    total_laser_flux = np.sum(injection_flux)
    #print(f"Total Laser Flux (scaled): {total_laser_flux:.6f} photons/sec")

    total_detected_pixels = 0
    for order_index, order in enumerate(orders):
        wl_obs = order["wavelength"].value
        obs_flux = order["flux"]
        obs_variance = order["variance"]
        model_flux = order["model_photon_flux"].value * median_eff_per_order[order_index]
        minimum_snr = constants_to_change()["minimum snr"]
        
        baseline_residual = obs_flux - model_flux
        #baseline_snr = baseline_residual / np.sqrt(obs_variance)

        laser_flux_interp = np.interp(wl_obs, shifted_wavelength_injection, injection_flux, left=0, right=0)
        laser_variance_interp = np.interp(wl_obs, shifted_wavelength_injection, injection_variance, left=0, right=0)

        injected_flux = obs_flux + laser_flux_interp
        injected_variance = obs_variance + laser_variance_interp

        residual_flux = injected_flux - model_flux
        residual_variance = injected_variance / np.sqrt(model_flux)
        snr = residual_flux / np.sqrt(residual_variance)
        
        
        #baseline = (obs_flux - model_flux) / np.sqrt(obs_variance)
        snr_excess = snr 
        laser_pixels = np.where(laser_flux_interp > 0)[0]
        #laser_pixels = np.where((wl_obs >= desired_laser_center.value - 2) & (wl_obs <= desired_laser_center.value + 2))[0]
        wl_laser_pixels = wl_obs[laser_pixels]
        snr_excess_laser = snr_excess[laser_pixels]
        total_detected_pixels += np.sum(snr_excess_laser >= minimum_snr)
        #total_detected_pixels += np.sum(snr >= minimum_snr)

        all_excess_snr.extend(snr_excess_laser)
        all_laser_wavelengths.extend(wl_laser_pixels)
        all_laser_excess.extend(snr_excess_laser >= minimum_snr)

        full_wavelength.extend(wl_obs)
        full_injected_flux.extend(injected_flux)
        full_injected_variance.extend(injected_variance)
        full_model_flux.extend(model_flux)
        full_residual.extend(residual_flux)
        full_snr.extend(snr)
        full_snr_excess.extend(snr_excess)

    # Sort results
    sorted_idx = np.argsort(full_wavelength)
    full_wavelength = np.array(full_wavelength)[sorted_idx]
    full_injected_flux = np.array(full_injected_flux)[sorted_idx]
    full_injected_variance = np.array(full_injected_variance)[sorted_idx]
    full_model_flux = np.array(full_model_flux)[sorted_idx]
    full_residual = np.array(full_residual)[sorted_idx]
    full_snr = np.array(full_snr)[sorted_idx]
    full_snr_excess = np.array(full_snr_excess)[sorted_idx]

    # print(f"Pixels with SNR ≥ {minimum_snr} above baseline: {total_detected_pixels}")
    # if total_detected_pixels >= 5:
    #     print("Laser Detected")
    # else:
    #     print("No Laser Detected")
    # # Residual Flux Plot
    # plt.figure(figsize=(12, 5))
    # plt.plot(full_wavelength, full_residual, label="Residual Flux (Injected - Model)", color='C3')
    # plt.xlabel("Wavelength (Å)")
    # plt.ylabel("Photons / sec")
    # plt.title("Residual Flux After Minimum Laser Injection (All Orders)")
    # plt.grid(True)
    # plt.legend()
    # plt.ylim(bottom=0)
    # ax = plt.gca()
    # ax.xaxis.set_major_formatter(ScalarFormatter())
    # ax.ticklabel_format(useOffset=False, style='plain', axis='x')

    # Optional: zoom into laser region (adjust limits as needed)
    window = 5  # Angstrom window around laser center to zoom

    x_min = desired_laser_center.value - window
    x_max = desired_laser_center.value + window

    # plt.xlim(x_min, x_max)
    # plt.tight_layout()
    # plt.show()

    # # SNR Excess Plot
    # plt.figure(figsize=(12, 5))
    # plt.plot(full_wavelength, full_snr_excess, label='SNR Above Baseline', color='purple')
    # plt.axhline(minimum_snr, color='red', linestyle='--', label=f'Threshold ({minimum_snr})')
    
    # # Highlight points where excess SNR ≥ 5
    # laser_excess_mask = np.array(all_laser_excess)
    # laser_wavelengths = np.array(all_laser_wavelengths)
    # laser_snr_excess = np.array(all_excess_snr)

    # plt.scatter(laser_wavelengths[laser_excess_mask],
    #             laser_snr_excess[laser_excess_mask],
    #             color='green', label=f'Pixels with SNR Excess ≥ {minimum_snr}')
    # plt.xlabel("Wavelength (Å)")
    # plt.ylabel("SNR Above Baseline")
    # plt.title("Excess SNR After Minimum Laser Injection")
    # plt.grid(True)
    # plt.legend()
    
    # ax = plt.gca()
    # ax.xaxis.set_major_formatter(ScalarFormatter())
    # ax.ticklabel_format(useOffset=False, style='plain', axis='x')
    # # Optional: zoom near laser center
    # plt.xlim(x_min, x_max)
    # plt.ylim(bottom=0)
    # plt.tight_layout()
    # plt.show()
    return (full_injected_flux, full_injected_variance, full_residual,
            full_snr, full_wavelength, total_laser_flux)

In [449]:
def loading_injecting_plotting(restframe, eff_func=None, folder_path=None, parq_file=None):
    orders = obs_spectrum(restframe)

    all_wavelengths = []
    all_efficiencies = []
    all_obs_flux = []
    all_variance = []
    order_centers = []
    median_eff_per_order = []

    for order in orders:
        obs_wavelengths = order["wavelength"]
        obs_flux = order["flux"]
        obs_variance = order.get("variance", None)

        # Model flux per order
        model_wavelength, model_photon_flux = plot_model_flux_with_obs_bins(parq_file, order)
        order["model_wavelength"] = model_wavelength
        order["model_photon_flux"] = model_photon_flux

        model_flux = model_photon_flux.value
        valid = (model_flux > 0) & (obs_flux > 0) & ~np.isnan(model_flux) & ~np.isnan(obs_flux)

        # Efficiency per order
        median_eff = np.median(obs_flux[valid] / model_flux[valid]) if np.any(valid) else np.nan
        median_eff_per_order.append(median_eff)
        order_centers.append(np.median(obs_wavelengths.value))

        if np.any(valid):
            all_wavelengths.append(obs_wavelengths.value[valid])
            all_efficiencies.append(obs_flux[valid] / model_flux[valid])
            all_obs_flux.append(obs_flux[valid])
            all_variance.append(obs_variance[valid] if obs_variance is not None else np.full(np.sum(valid), np.nan))

    # Combine arrays
    all_wavelengths = np.concatenate(all_wavelengths)
    all_efficiencies = np.concatenate(all_efficiencies)
    all_obs_flux = np.concatenate(all_obs_flux)
    all_variance = np.concatenate(all_variance)
    median_eff_per_order = np.nan_to_num(median_eff_per_order, nan=0.0)

    # Inject laser flux and variance using lfc
    wavelength_injection, injection_flux = injection_laser_flux(lfc)
    wavelength_variance, injection_variance = injection_laser_variance(lfc)

    full_injected_flux, full_injected_variance, full_residual, full_snr, full_wavelength, total_laser_flux = inject_and_analyze_laser(
        orders, injection_flux, injection_variance, wavelength_injection, median_eff_per_order=median_eff_per_order
    )

    # Get laser center from constants_to_change
    laser_center = constants_to_change()["laser center"]

    return laser_center, total_laser_flux, all_wavelengths, all_efficiencies

In [450]:


def compute_efficiency_interpolation_from_folder(folder_path, model_path, plot=True):
    fits_files = sorted(glob.glob(f"{folder_path}/*.fits"))
    if not fits_files:
        raise FileNotFoundError(f"No .fits files found in folder: {folder_path}")

    wavelength_efficiencies = defaultdict(list)
    model_flux_cache = {}  # key: order number or binning structure

    for path in fits_files:
        try:
            orders = obs_spectrum(path)  # list of dicts

            for order in orders:
                order_id = order.get("order", None)
                obs_wavelengths = order["wavelength"]
                obs_flux = order["flux"]
                obs_binwidths = order["binwidths"]

                # Use order ID or wavelength bin as cache key
                cache_key = tuple(obs_wavelengths.to_value(u.AA))  # hashable key
                if cache_key in model_flux_cache:
                    model_wavelengths, model_photon_flux = model_flux_cache[cache_key]
                else:
                    model_wavelengths, model_photon_flux = plot_model_flux_with_obs_bins(model_path, order)
                    model_flux_cache[cache_key] = (model_wavelengths, model_photon_flux)

                valid = (obs_flux > 0) & (model_photon_flux > 0)
                if np.sum(valid) == 0:
                    continue

                efficiency = (obs_flux[valid] / model_photon_flux[valid]).value
                wavelengths = obs_wavelengths[valid].value

                for wl, eff in zip(wavelengths, efficiency):
                    wavelength_efficiencies[wl].append(eff)

        except Exception as e:
            warnings.warn(f"Skipping {path} due to error: {e}")
            continue

    all_wavelengths = np.array(sorted(wavelength_efficiencies.keys()))
    all_avg_efficiencies = np.array([np.mean(wavelength_efficiencies[wl]) for wl in all_wavelengths])

    if plot:
        plt.figure(figsize=(10, 5))
        plt.plot(all_wavelengths, all_avg_efficiencies, label="Average Efficiency", color='tab:blue')
        plt.xlabel("Wavelength (Å)")
        plt.ylabel("Efficiency")
        plt.title("Mean Instrument Efficiency Across All Spectra")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

    print(f"Loaded {len(all_wavelengths)} wavelengths and {len(all_avg_efficiencies)} efficiencies")

    if len(all_wavelengths) == 0 or len(all_avg_efficiencies) == 0:
        raise ValueError("Empty data arrays, cannot create interpolation")

    interp_eff_func = interp1d(
        all_wavelengths,
        all_avg_efficiencies,
        kind='linear',
        bounds_error=False,
        fill_value=0.0
    )

    return interp_eff_func

In [451]:
#finds the minimum power required for detection for laser injected at every pixel

def scan_minimum_power_vs_wavelength(restframe_path, parq_file, lfc_file, folder_path, step=1, eff_func=None):
    orders = obs_spectrum(restframe_path)
    wavelength_template, flux_template = injection_laser_flux(lfc_file)
    _, var_template = injection_laser_variance(lfc_file)

    # Preload data once
    laser_center, total_flux, _, _ = loading_injecting_plotting(
    restframe_path, eff_func=eff_func, folder_path=folder_path, parq_file=parq_file
)
    _, _, all_wavelengths, _ = loading_injecting_plotting(
        restframe_path, eff_func=eff_func, folder_path=folder_path, parq_file=parq_file
)

    # Compute efficiency interpolation function once
    eff_func = compute_efficiency_interpolation_from_folder(folder_path, parq_file, plot=False)
    telescope_radius = constants_to_change()["telescope radius"]  # in meters
    A_tel = np.pi * (telescope_radius)**2
    transmitter_diameter = constants_to_change()["transmitter diameter"]  # in meters
    distance = constants_to_change()["distance to star"].to(u.m)

    results_wavelengths = []
    required_powers_kW = []

    for idx in tqdm(range(0, len(all_wavelengths), step), desc="Scanning wavelengths"):
        center = all_wavelengths[idx] * u.Angstrom

        efficiency = eff_func(center.value)
        if efficiency <= 0:
            efficiency = 1e-10  # avoid division by zero or negative efficiencies

        E_photon = (c.h * c.c / center).to(u.J)
        photon_flux = total_flux * (1 / u.s)

        P_watts = (E_photon * photon_flux)

        flux_at_detector = (P_watts / A_tel)
        flux_at_earth = (flux_at_detector / efficiency)

        theta = (center / transmitter_diameter).decompose()
        Omega = (np.pi * theta**2 / 4).decompose()

        eirp = (flux_at_earth * (4 * np.pi * distance**2)).to(u.W)
        emitted_power = (eirp * Omega).to(u.kW)

        results_wavelengths.append(center.value)
        required_powers_kW.append(emitted_power.value)

    return np.array(results_wavelengths), np.array(required_powers_kW)

In [452]:
#injects the specified power of laser to the observed spectra
#plots snr and residual flux, also says if laser was detected or not (if 5 pixels are above the SNR specified)
#returns flux, variance, snr, residual flux, and wavelength of injection
def inject_specified_power(restframe_path, desired_power_kW, laser_center):
    
    # Load observed spectrum
    orders = obs_spectrum(restframe_path)

    # Compute model photon flux
    for order in orders:
        model_wavelength, model_photon_flux = plot_model_flux_with_obs_bins(parq_file, order)
        order["model_wavelength"] = model_wavelength
        order["model_photon_flux"] = model_photon_flux

    # Calculate median efficiencies per order
    median_eff_per_order = []
    for order in orders:
        valid = (order["model_photon_flux"].value > 0) & (order["flux"] > 0)
        median_eff = np.median(order["flux"][valid] / order["model_photon_flux"].value[valid]) if np.any(valid) else 0
        median_eff_per_order.append(median_eff)

    # Load laser template
    wavelength_injection, injection_flux = injection_laser_flux(lfc)
    wavelength_variance, injection_variance = injection_laser_variance(lfc)

    # Constants
    laser_center *= u.Angstrom
    distance_to_star = constants_to_change()["distance to star"].to(u.m)
    telescope_diameter = 8.5 * u.m  # HPF collecting aperture (adjust if needed)
    telescope_area = np.pi * (telescope_diameter / 2)**2

    # Beam divergence
    transmitter_diameter = constants_to_change()["transmitter diameter"]   # Diameter of the transmitting telescope
    
    wavelength = laser_center
    theta = (wavelength / transmitter_diameter).decompose()
    Omega = (np.pi * theta**2 / 4).decompose()

    # Laser emission power
    P_laser = desired_power_kW 

    # Compute EIRP
    eirp = (P_laser / Omega)

    # Flux at Earth (top of atmosphere)
    flux_at_earth = (eirp / (4 * np.pi * distance_to_star**2))
    
    # Get efficiency at laser wavelength
    wavelength, photons_per_sec, all_wavelengths, all_efficiencies = loading_injecting_plotting(restframe)
    wavelength*=u.Angstrom # center of injection
    wavelength_array = all_wavelengths * u.Angstrom  # ensure units are consistent
    idx = np.argmin(np.abs(wavelength_array - wavelength))
    folder = folder_path
    eff_func = compute_efficiency_interpolation_from_folder(folder_path, parq_file)
    efficiency = eff_func(wavelength.value)
    # Flux at detector
    flux_at_detector = flux_at_earth * efficiency

    # Power at detector
    P_detector = flux_at_detector * telescope_area

    # Photon energy
    E_photon = (c.h * c.c / wavelength)

    # Photon flux at detector (photons/sec)
    photon_flux_at_detector = (P_detector / E_photon).to(1 / u.s)

    # Scale laser template to match this photon flux
    template_total_flux = np.sum(injection_flux)  # photons/sec (template units)
    scale_factor = photon_flux_at_detector.value / template_total_flux

    print(f"\nInjecting laser with emitted power: {desired_power_kW:.2f}")
    print(f"Scale factor: {scale_factor:.4g}")
    print(f"Instrument efficiency at {wavelength}: {efficiency:.4f}")
    print(f"Estimated flux received at {wavelength} with injected power of laser power of {desired_power_kW:.2f}: {photon_flux_at_detector:.4g}:")
    # Scale injection
    scaled_injection_flux = injection_flux * scale_factor
    scaled_injection_variance = injection_variance * scale_factor

    # Shift laser to desired center
    original_laser_peak = wavelength_injection[np.argmax(injection_flux)] * u.Angstrom
    shift_angstroms = (laser_center - original_laser_peak).value
    shifted_wavelength_injection = wavelength_injection + shift_angstroms

    # Inject into each order
    full_wavelength = []
    full_injected_flux = []
    full_injected_variance = []
    full_model_flux = []
    full_residual = []
    full_snr = []
    full_snr_excess = []

    for order_index, order in enumerate(orders):
        wl_obs = order["wavelength"].value
        obs_flux = order["flux"]
        obs_variance = order["variance"]
        model_flux = order["model_photon_flux"].value * median_eff_per_order[order_index]

        laser_flux_interp = np.interp(wl_obs, shifted_wavelength_injection, scaled_injection_flux, left=0, right=0)
        laser_var_interp = np.interp(wl_obs, shifted_wavelength_injection, scaled_injection_variance, left=0, right=0)

        injected_flux = obs_flux + laser_flux_interp
        injected_variance = obs_variance + laser_var_interp

        residual_flux = injected_flux - model_flux
        residual_variance = injected_variance / np.sqrt(model_flux)
        snr = residual_flux / np.sqrt(residual_variance)
        baseline = (obs_flux - model_flux) / np.sqrt(obs_variance)
        snr_excess = snr 

        full_wavelength.extend(wl_obs)
        full_injected_flux.extend(injected_flux)
        full_injected_variance.extend(injected_variance)
        full_model_flux.extend(model_flux)
        full_residual.extend(residual_flux)
        full_snr.extend(snr)
        full_snr_excess.extend(snr_excess)

    # Sort all arrays by wavelength
    sorted_idx = np.argsort(full_wavelength)
    full_wavelength = np.array(full_wavelength)[sorted_idx]
    full_injected_flux = np.array(full_injected_flux)[sorted_idx]
    full_injected_variance = np.array(full_injected_variance)[sorted_idx]
    full_model_flux = np.array(full_model_flux)[sorted_idx]
    full_residual = np.array(full_residual)[sorted_idx]
    full_snr = np.array(full_snr)[sorted_idx]
    full_snr_excess = np.array(full_snr_excess)[sorted_idx]

    # Plot residual flux
    window = 2  # Angstrom
    plt.figure(figsize=(8, 4))
    plt.plot(full_wavelength, full_residual, label="Residual Flux", color='C3')
    plt.xlabel("Wavelength (Å)")
    plt.ylabel("Photons / sec")
    plt.title(f"Residual Flux After {desired_power_kW:.2f} Laser Injection")
    plt.grid(True); plt.legend(); plt.ylim(bottom=0)
    plt.xlim(laser_center.value - window, laser_center.value + window)
    ax = plt.gca()
    ax.xaxis.set_major_formatter(ScalarFormatter())
    ax.ticklabel_format(useOffset=False, style='plain', axis='x')
    #ax.ticklabel_format(useOffset=False, style='plain', axis='y')
    plt.tight_layout(); plt.show()


    minimum_snr = constants_to_change()["minimum snr"]
    #Plot SNR excess
    plt.figure(figsize=(8, 4))
    plt.plot(full_wavelength, full_snr_excess, label='S/N Above Baseline', color='purple')
    plt.axhline(5, color='red', linestyle='--', label=f'Threshold ({minimum_snr})')

    # Find pixels with SNR excess ≥ 5
    snr_mask = full_snr_excess >= minimum_snr
    num_detected_pixels = np.sum(snr_mask)
    # Define pixel window around laser center
    pixel_width = np.median(np.diff(full_wavelength))  # assume roughly uniform pixel spacing
    half_width = 2  # pixels on either side of center
    center_window = ((full_wavelength >= laser_center.value - half_width * pixel_width) &
                    (full_wavelength <= laser_center.value + half_width * pixel_width))

    # Check if those specific 5 pixels are above SNR threshold
    snr_in_window = full_snr_excess[center_window]
    num_pixels_above_threshold = np.sum(snr_in_window >= minimum_snr)

    if num_pixels_above_threshold == 5:
        print("Laser detected based on 5 central pixels!")
    else:
        print("No laser detected in central 5-pixel window.")
        
    # Scatter plot detected pixels
    plt.scatter(np.array(full_wavelength)[snr_mask],
                np.array(full_snr_excess)[snr_mask],
                color='green', label=f'Pixels with S/N Excess ≥ {minimum_snr}')
    # Overlay highlight on the central 5 pixels
    plt.scatter(np.array(full_wavelength)[center_window],
                np.array(full_snr_excess)[center_window],
                color='orange', edgecolor='black', label='Laser Injection Region', zorder=5)
    plt.xlabel("Wavelength (Å)")
    plt.ylabel("S/N Above Baseline")
    plt.title(f"S/N Excess After {desired_power_kW:.2f} Laser Injection")
    plt.grid(True)
    plt.legend()
    plt.xlim(laser_center.value - window, laser_center.value + window)
    plt.ylim(bottom=0)
    ax = plt.gca()
    ax.xaxis.set_major_formatter(ScalarFormatter())
    ax.ticklabel_format(useOffset=False, style='plain', axis='x')
    plt.tight_layout()
    plt.show()
    
    # Define pixel window around laser center
    pixel_width = np.median(np.diff(full_wavelength))  # assume roughly uniform pixel spacing
    half_width = 2  # pixels on either side of center
    center_window = ((full_wavelength >= laser_center.value - half_width * pixel_width) &
                    (full_wavelength <= laser_center.value + half_width * pixel_width))

    # Check if those specific 5 pixels are above SNR threshold
    snr_in_window = full_snr_excess[center_window]
    num_pixels_above_threshold = np.sum(snr_in_window >= minimum_snr)

    if num_pixels_above_threshold == 5:
        print("Laser detected based on 5 central pixels!")
    else:
        print("No laser detected in central 5-pixel window.")
        
    return (full_injected_flux, full_injected_variance, full_residual,
            full_snr, full_wavelength, np.sum(scaled_injection_flux))


In [453]:
def laser_gaussian_profile(center_wl, total_counts_per_s, pixel_width, fwhm_pixels=1.65, efficiency_func=None):
    fwhm = fwhm_pixels * pixel_width
    sigma = fwhm / (2 * np.sqrt(2 * np.log(2)))

    wl = np.linspace(center_wl - 5 * sigma, center_wl + 5 * sigma, 1000)
    amplitude = total_counts_per_s / (sigma * np.sqrt(2 * np.pi))
    flux = amplitude * np.exp(-0.5 * ((wl - center_wl) / sigma)**2)

    if efficiency_func:
        eff_array = efficiency_func(wl * u.AA)
        flux *= eff_array
    else:
        eff_array = np.ones_like(wl)

    photon_energy = (c.h * c.c / (center_wl * u.AA)).to(u.J)
    power = (total_counts_per_s / u.s * photon_energy).to(u.W)

    return wl, flux, power.value, eff_array

# === POWER ESTIMATION ===
def estimate_emitted_laser_power(wavelength_angstrom, photons_per_sec, efficiency, telescope_radius, transmitter_diameter, distance_to_star_pc):
    import astropy.units as u
    import numpy as np

    wavelength = wavelength_angstrom * u.Angstrom
    photons_per_sec = photons_per_sec * (1 / u.s)
    dist_pc = distance_to_star_pc if isinstance(distance_to_star_pc, u.Quantity) else distance_to_star_pc * u.pc

    # Energy per photon
    E_photon = (c.h * c.c / wavelength)

    # Power detected at telescope
    P_detected = (E_photon * photons_per_sec)

    # Telescope collecting area (in m^2)
    A_collect = (np.pi * telescope_radius**2)

    # Flux at telescope (W/m²)
    flux_at_detector = (P_detected / A_collect)

    # Correct for instrument efficiency (unitless)
    flux_at_earth = (flux_at_detector / efficiency)

    # Beam divergence angle (radians)
    theta = (wavelength / transmitter_diameter)

    # Solid angle of beam (steradians, dimensionless)
    Omega = (np.pi * theta**2 / 4).decompose()

    # Convert distance to meters
    dist_m = dist_pc

    # Total emitted power (W)
    emitted_power = (flux_at_earth * Omega * dist_m**2)

    # Convert to megawatts
    emitted_power_mw = emitted_power.to(u.MW)

    # Print summary
    print(f"Target wavelength: {wavelength:.4f}")
    print(f"Efficiency at detector: {efficiency:.4f}")
    print(f"Photon flux at detector: {photons_per_sec:.4g}")
    print(f"Estimated emitted laser power: {emitted_power_mw:.3f}")

    return emitted_power_mw

# === UTILITIES ===
def get_valid_wavelength_ranges(orders):
    wmins, wmaxs = [], []
    for order in orders:
        w = order['wavelength']
        wmins.append(w.min().to(u.AA).value)
        wmaxs.append(w.max().to(u.AA).value)
    return np.array(wmins), np.array(wmaxs)

def filter_wavelengths(wavelengths, wmins, wmaxs):
    mask = np.zeros_like(wavelengths, dtype=bool)
    for wmin, wmax in zip(wmins, wmaxs):
        mask |= (wavelengths >= wmin) & (wavelengths <= wmax)
    return mask

# === MAIN ANALYSIS ===
'AT THE TOP OF THIS FUNCTION, TOTAL_COUNTS IS USED TO CHANGE THE AMPLITUDE OF GAUSSIAN AND DETERMINE HOW MANY COUNTS ARE DETECTED'
def analyze_laser_detectability(restframe, parq_file, folder_path, constants_to_change, scan_minimum_power_vs_wavelength):


    # Setup laser center wavelength with units
    laser_center = constants_to_change()["laser center"] * u.Angstrom
    total_counts = 5 # target photon counts per second. power will be calculated for this value

    # Load efficiency function once
    eff_func = compute_efficiency_interpolation_from_folder(folder_path, parq_file, plot=False)

    # Pass eff_func to loading_injecting_plotting to avoid reloading
    laser_center_unused, total_laser_flux, all_wavelengths, all_efficiencies = loading_injecting_plotting(
        restframe, eff_func=eff_func, folder_path=folder_path, parq_file=parq_file
    )

    # Aggregate all wavelengths and get pixel width
    orders = obs_spectrum(restframe)
    all_wavelengths_full = np.concatenate([order["wavelength"].value for order in orders if len(order["wavelength"]) > 1])
    pixel_width = np.median(np.diff(np.sort(all_wavelengths_full)))
    print(f"Pixel width (Å): {pixel_width:.4f}")

    # Gaussian laser profile simulation (using the efficiency function)
    wavelengths, fluxes, power, efficiency = laser_gaussian_profile(
        laser_center.value, total_counts, pixel_width, efficiency_func=eff_func
    )

    # Get injected laser data and corresponding efficiencies
    wavelength, photons_per_sec, all_wavelengths_injected, all_efficiencies_injected = loading_injecting_plotting(restframe, eff_func=eff_func, folder_path=folder_path, parq_file=parq_file)

    total_counts *= 1 / u.s
    wavelength *= u.Angstrom
    wavelength_array = all_wavelengths_injected * u.Angstrom
    idx = np.argmin(np.abs(wavelength_array - wavelength))

    # Compute efficiency at injected wavelength
    efficiency_at_wavelength = eff_func(wavelength.value)

    # Energy per photon (Joules)
    E_photon = c.h * c.c / wavelength

    # Power at detector (Watts)
    P_watts = E_photon * total_counts

    # Telescope radius from constants
    hpf_radius = constants_to_change()["telescope radius"]

    # Flux at detector (W/m^2)
    flux_at_detector = P_watts / (np.pi * hpf_radius**2)

    # Flux at Earth's atmosphere top corrected for efficiency
    flux_at_earth = flux_at_detector / efficiency_at_wavelength

    # Beam divergence and solid angle
    tx_diam = constants_to_change()["transmitter diameter"]
    theta = wavelength / tx_diam
    Omega = (np.pi * theta**2) / 4

    # Distance to star in meters
    dist_pc = constants_to_change()["distance to star"]
    dist_m = dist_pc.to(u.m)

    # Effective isotropic radiated power (EIRP)
    eirp = flux_at_earth * (4 * np.pi * dist_m**2)

    # Actual emitted power (MW)
    power = (eirp * Omega).to(u.MW)

    print(f"\nTarget wavelength: {wavelength:.4f}")
    print(f"Closest wavelength in data: {wavelength_array[idx]:.4f}")
    print(f"Efficiency at that wavelength: {efficiency_at_wavelength:.4f}")
    print(f"Estimated emitted laser power at {wavelength:.4f} with detected flux of {total_counts:.4g}: {power:.4f}")

    # # Minimum power scan across wavelengths
    # wmins, wmaxs = get_valid_wavelength_ranges(orders)
    # wavelengths_scan, powers_scan = scan_minimum_power_vs_wavelength(
    # restframe, parq_file, lfc, folder_path, step=1, eff_func=eff_func)
    # wavelengths_scan = np.array([wl.to(u.Angstrom).value if hasattr(wl, 'to') else wl for wl in wavelengths_scan])

    # mask = filter_wavelengths(wavelengths_scan, wmins, wmaxs)
    # filtered_wavelengths = wavelengths_scan[mask]
    # filtered_powers = powers_scan[mask]

    # print(f'Min laser power: {np.min(filtered_powers):.2e} kW')
    # print(f'Max laser power: {np.max(filtered_powers):.2e} kW')

    # filename = os.path.basename(restframe)

    # # Clip outliers for plotting
    # lower, upper = np.percentile(filtered_powers, [5, 95])
    # clip_mask = (filtered_powers >= lower) & (filtered_powers <= upper)
    # plot_wl = filtered_wavelengths[clip_mask]
    # plot_pw = filtered_powers[clip_mask]

    # rolling_10, rolling_90, roll_wl = [], [], []
    # window = 601

    # for wmin, wmax in zip(wmins, wmaxs):
    #     order_mask = (plot_wl >= wmin) & (plot_wl <= wmax)
    #     wl_order = plot_wl[order_mask]
    #     pw_order = plot_pw[order_mask]

    #     if len(pw_order) < window:
    #         q10 = pd.Series(pw_order).quantile(0.10)
    #         q90 = pd.Series(pw_order).quantile(0.90)
    #         r10 = np.full_like(pw_order, q10)
    #         r90 = np.full_like(pw_order, q90)
    #     else:
    #         s = pd.Series(pw_order)
    #         r10 = s.rolling(window, center=True, min_periods=1).quantile(0.10).values
    #         r90 = s.rolling(window, center=True, min_periods=1).quantile(0.90).values

    #     rolling_10.append(r10)
    #     rolling_90.append(r90)
    #     roll_wl.append(wl_order)

    # # Plot results
    # import matplotlib.pyplot as plt

    # fig, ax = plt.subplots(figsize=(14, 6))
    # ax.scatter(plot_wl, plot_pw, color='powderblue', s=2.5, label="Required laser power (5–95%)")
    # ax.plot(np.concatenate(roll_wl), np.concatenate(rolling_10), color='red', lw=1.5, label="Rolling 10th percentile")
    # ax.plot(np.concatenate(roll_wl), np.concatenate(rolling_90), color='green', lw=1.5, label="Rolling 90th percentile")

    # ax.set_xlabel("Wavelength (Å)")
    # ax.set_ylabel("Minimum Laser Power (kW)")
    # ax.set_title(f"Detectable Laser Power vs. Wavelength\n{filename}")
    # ax.grid(True)
    # ax.legend(loc='upper right')

    # text = (f"Min Power (5th percentile): {np.min(plot_pw):.2f} kW\n"
    #         f"Max Power (95th percentile): {np.max(plot_pw):.2f} kW")
    # fig.text(0.31, 0.81, text, fontsize=11, bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

    # plt.show()

analyze_laser_detectability(restframe, parq_file, folder_path, constants_to_change, scan_minimum_power_vs_wavelength)

Loaded 778215 wavelengths and 778215 efficiencies
Pixel width (Å): 0.0621

Target wavelength: 10010.0000 Angstrom
Closest wavelength in data: 10010.0149 Angstrom
Efficiency at that wavelength: 0.0227
Estimated emitted laser power at 10010.0000 Angstrom with detected flux of 5 1 / s: 0.5415 MW


In [454]:
'all average pixel efficiencies plotted'

# # Track results
# all_wavelengths = []
# all_efficiencies = []
# median_eff_per_order = []
# order_centers = []

# fits_files = [f for f in os.listdir(folder_path) if f.endswith(".fits")]
# print(f"Found {len(fits_files)} FITS files")
# eff_per_order = defaultdict(list)
# wavelengths_per_order = defaultdict(list)

# for file in fits_files:
#     filepath = os.path.join(folder_path, file)
#     try:
#         orders = obs_spectrum(filepath)
#     except Exception as e:
#         print(f"Could not read {file}: {e}")
#         continue

#     for order in orders:
#         try:
#             obs_wavelengths = order["wavelength"]
#             obs_flux = order["flux"]
#             order_number = order["order"]

#             model_wavelength, model_photon_flux = plot_model_flux_with_obs_bins(parq_file, order)
#             model_flux = model_photon_flux.value

#             valid = (model_flux > 0) & (obs_flux > 0) & ~np.isnan(model_flux) & ~np.isnan(obs_flux)

#             if np.any(valid):
#                 all_wavelengths.append(obs_wavelengths.value[valid])
#                 all_efficiencies.append(obs_flux[valid] / model_flux[valid])
#                 eff_per_order[order_number].extend((obs_flux[valid] / model_flux[valid]).tolist())
#                 wavelengths_per_order[order_number].extend(obs_wavelengths.value[valid].tolist())

#         except Exception as e:
#             print(f"⚠️ Skipping order due to error: {e}")
#             continue

# # Finalize arrays
# if not all_wavelengths:
#     raise ValueError("No valid wavelengths found in any FITS files.")

# all_wavelengths = np.concatenate(all_wavelengths)
# all_efficiencies = np.concatenate(all_efficiencies)

# # Compute median efficiency and wavelength per order
# median_eff_per_order = []
# order_centers = []

# for order in sorted(eff_per_order.keys()):
#     median_eff = np.median(eff_per_order[order])
#     center_wav = np.median(wavelengths_per_order[order])
#     median_eff_per_order.append(median_eff)
#     order_centers.append(center_wav)

# # Plot
# plt.figure(figsize=(15, 5))
# plt.scatter(all_wavelengths, all_efficiencies, s=2, label="Efficiency (Obs / Model)")
# plt.plot(order_centers, median_eff_per_order, 'ro-', label="Median per Order", linewidth=1.5)
# plt.xlabel("Wavelength (Å)")
# plt.ylabel("Efficiency")
# plt.title("Instrument Efficiency vs Wavelength")
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

'all average pixel efficiencies plotted'